In [2]:
#Traditional neural networks can’t do this, and it seems like a major shortcoming. For example, imagine you want to 
#classify what kind of event is happening at every point in a movie. It’s unclear how a traditional neural 
#network could use its reasoning about previous events in the film to inform later ones.

#Recurrent neural networks address this issue. They are networks with loops in them, allowing information to persist.

#LSTMs are explicitly designed to avoid the long-term dependency problem. 
#Remembering information for long periods of time is practically their default behavior, 
#not something they struggle to learn!

# Best Read http://colah.github.io/posts/2015-08-Understanding-LSTMs/

# THE BELOW PROBLEM CAN BE SOLVED USING CNN ALSO. 
# But it is worth to solve using LSTM.



In [3]:
# Decalring Model Parameters
batch_size = 4096
STROKE_COUNT = 196
TRAIN_SAMPLES = 750
VALID_SAMPLES = 75
TEST_SAMPLES = 50

In [4]:
# Importing standard libraries 
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd

Using TensorFlow backend.


In [5]:

#A metric is a function that is used to judge the performance of your model. 
#Metric functions are to be supplied in the metrics parameter when a model is compiled.

from keras.metrics import top_k_categorical_accuracy

#Available metrics
#binary_accuracy , categorical_accuracy, sparse_categorical_accuracy, top_k_categorical_accuracy, 
#sparse_top_k_categorical_accuracy

#AND 

#Custom Metrics

In [6]:
def top_3_accuracy(x,y): return top_k_categorical_accuracy(x,y, 3)

In [7]:
#A callback is a set of functions to be applied at given stages of the training procedure. 
#You can use callbacks to get a view on internal states and statistics of the model during training. 
#You can pass a list of callbacks (as the keyword argument callbacks) to the .fit() method 
#of the Sequential or Model classes. The relevant methods of the callbacks will then be called 
#at each stage of the training.

# ModelCheckpoint - Save the model after every epoch.
# LearningRateScheduler - Learning rate scheduler.
# EarlyStopping - top training when a monitored quantity has stopped improving.
# ReduceLROnPlateau - Reduce learning rate when a metric has stopped improving.

from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from glob import glob
import gc
gc.enable()

In [8]:
# Checking for available GPU, but for our learnings lets skip this.
def get_available_gpus():
    from tensorflow.python.client import device_lib
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [9]:
base_dir = '..\..\data\doodleoutput'
base_dir

'..\\..\\data\\doodleoutput'